# Graph methods for imaging, Vision, and computing (B31RX) 2025

## Tutorial 8: Variational Autoencoders (VAEs)

In this tutorial, we will explore the application of variational autoencoders (VAEs), a type of generative model capable of learning the underlying structure of data, for denoising images, specifically using the MNIST dataset of handwritten digits.

---

### Background

Image denoising is an essential preprocessing step in many computer vision and image processing applications. The goal is to remove noise from a given image while preserving the original details and structures as much as possible. Traditional denoising techniques, such as Gaussian filtering or total variation regularization, often struggle to balance noise removal and detail preservation.

In recent years, deep learning techniques, including autoencoders and their variants, have shown great potential in denoising tasks. Variational Autoencoders (VAEs) are one such deep learning model, primarily known for their generative capabilities.

- VAEs consist of an encoder network that maps input data to a latent space and a decoder network that reconstructs the input data from the latent space.
- By optimizing both the reconstruction loss and a regularization term that encourages the latent space to follow a specific distribution, VAEs can learn smooth and structured latent spaces, which can be advantageous for denoising tasks.

In this tutorial, we will adapt VAEs for image denoising using the MNIST dataset of handwritten digits. We will explore various aspects of VAEs in the context of denoising, such as:
- The effects of noise levels, latent space dimensions, network architecture, and loss functions.

By studying these factors, we aim to provide insights into the effective application of VAEs for denoising tasks and their potential improvements.

### Question 1

Execute the provided code for the VAE with a dense-layer architecture. In the context of the given implementation, explain how Variational Autoencoders (VAEs) differ from traditional autoencoders.

In [55]:
# Note: Some parts of this code (such as the VAE architecture and Sampling layer) are adapted from the official
# Keras VAE tutorial: https://keras.io/examples/generative/vae/

import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras
from keras import ops
from keras import layers
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D  # ensures 3D plotting support
from sklearn.decomposition import PCA

# --------------------------------------
# Define the Sampling Layer
# --------------------------------------
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.seed_generator = keras.random.SeedGenerator(1337)

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = ops.shape(z_mean)[0]
        dim = ops.shape(z_mean)[1]
        epsilon = keras.random.normal(shape=(batch, dim), seed=self.seed_generator)
        return z_mean + ops.exp(0.5 * z_log_var) * epsilon

# --------------------------------------
# Define the VAE Model with a Custom Train Step
# --------------------------------------
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.current_epoch = 0  # initialize current epoch

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        # Unpack the tuple: noisy input and clean target
        noisy, clean = data
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(noisy)
            reconstruction = self.decoder(z)
            # Compute the reconstruction loss against the clean image
            reconstruction_loss = ops.mean(
                ops.sum(
                    keras.losses.binary_crossentropy(clean, reconstruction),
                    axis=(1, 2),
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - ops.square(z_mean) - ops.exp(z_log_var))
            kl_loss = ops.mean(ops.sum(kl_loss, axis=1))
            # Pseudo-code for annealing beta:
            # current_beta = min(1.0, beta + (self.current_epoch / num_epochs))
            total_loss = reconstruction_loss + beta * kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

class EpochTracker(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        self.model.current_epoch = epoch

In [ ]:
latent_dim = 3
num_epochs = 50
beta = 1
noise_stddev = 0.0

# --------------------------------------
# Build the Dense Encoder
# --------------------------------------
encoder_inputs = keras.Input(shape=(28, 28, 1), name="encoder_input")
x = layers.Flatten()(encoder_inputs)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dense(256, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

# --------------------------------------
# Build the Dense Decoder
# --------------------------------------
latent_inputs = keras.Input(shape=(latent_dim,), name="z_sampling")
x = layers.Dense(256, activation="relu")(latent_inputs)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dense(28 * 28, activation="sigmoid")(x)
decoder_outputs = layers.Reshape((28, 28, 1))(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

# --------------------------------------
# Load and Preprocess MNIST Data
# --------------------------------------
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
# Normalize and add channel dimension
x_train = np.expand_dims(x_train[:10000], -1).astype("float32") / 255.0
x_test  = np.expand_dims(x_test, -1).astype("float32") / 255.0
y_train = y_train[:10000]

def add_gaussian_noise(x, stddev=0.0):
    noise = np.random.normal(loc=0.0, scale=stddev, size=x.shape)
    x_noisy = x + noise
    # Clip values to keep them in [0, 1]
    return np.clip(x_noisy, 0., 1.)

# Add noise with a chosen standard deviation
x_train_noisy = add_gaussian_noise(x_train, stddev=noise_stddev)
x_test_noisy  = add_gaussian_noise(x_test, stddev=noise_stddev)

# --------------------------------------
# Create and Train the VAE
# --------------------------------------
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(x_train_noisy, x_train, epochs=num_epochs, batch_size=100, callbacks=[EpochTracker()])

### Question 2: Using the VAE for dimensionality reduction

Utilize the VAE to perform dimensionality reduction. Compare the latent space representations produced by the VAE for latent dimensions of 2 and 3 with those obtained using principal component analysis (PCA) applied to the raw data. Which method provides a more accurate and meaningful representation of the data?

Study and discuss the impact of changing the beta parameter (the regularization constant for the KL divergence term) on the structure of the latent space.

### Question 3: Using the VAE as a generator

Utilize the trained VAE's generative capabilities to synthesize new, realistic handwritten digits that are not part of the original MNIST dataset.

Study and discuss the impact of changing the beta parameter and the latent dimension on the quality, diversity, and realism of the generated digits.


### Question 4: Using the VAE for denoising

Add Gaussian noise to the input data. Investigate the effect of varying levels of noise on the performance of a VAE in denoising MNIST images. How does the VAE adapt to different noise levels, and what are the limitations?

Study and discuss the impact of changing the beta parameter and the latent space dimension on the quality of the denoised reconstructions.



### Question 5

Examine the impact of different types of noise (e.g., salt-and-pepper, or speckle noise) on the VAE's denoising performance for MNIST images. How does the VAE handle various noise types, and are there specific noise types that pose more significant challenges?

In [ ]:
def add_gaussian_noise(x, stddev=0.0):
    noise = np.random.normal(loc=0.0, scale=stddev, size=x.shape)
    x_noisy = x + noise
    # Clip values to keep them in [0, 1]
    return np.clip(x_noisy, 0., 1.)

def add_salt_and_pepper_noise(x, salt_prob=0.5, pepper_prob=0.1):
    """
    Adds salt and pepper noise to an image.

    Parameters:
    - x: Input image array with values in [0, 1].
    - salt_prob: Probability of adding salt (white pixels).
    - pepper_prob: Probability of adding pepper (black pixels).

    Returns:
    - Noisy image with some pixels set to 0 (pepper) or 1 (salt).
    """
    x_noisy = np.copy(x)
    random_matrix = np.random.rand(*x.shape)
    # Pepper noise: set pixels to 0
    x_noisy[random_matrix < pepper_prob] = 0.0
    # Salt noise: set pixels to 1
    x_noisy[random_matrix > 1 - salt_prob] = 1.0
    return x_noisy

def add_speckle_noise(x, stddev=0.1):
    """
    Adds speckle noise (multiplicative noise) to an image.

    Parameters:
    - x: Input image array with values in [0, 1].
    - stddev: Standard deviation of the noise.

    Returns:
    - Noisy image computed as x + x * noise.
    """
    noise = np.random.randn(*x.shape) * stddev
    x_noisy = x + x * noise
    return np.clip(x_noisy, 0., 1.)

### Question 6

Enhance the VAE architecture by replacing the dense encoder/decoder with convolutional blocks.

Study and discuss the impact of this architectural change on dimensionality reduction, generation of new digits, and denoising performance.